In [ ]:
import os
import re
import json
import requests
from tqdm import tqdm

# -----------------------------
# Función para obtener el secUid
# -----------------------------
def get_seguid(username):
    url = f"https://www.tiktok.com/@{username}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0 Safari/537.36"
    }
    r = requests.get(url, headers=headers)
    if r.status_code != 200:
        raise Exception(f"Error al obtener perfil: {r.status_code}")

    match = re.search(r'"secUid":"(.*?)"', r.text)
    if not match:
        raise Exception("No se pudo extraer secUid (TikTok cambió HTML o usuario no existe)")
    return match.group(1)

# -----------------------------
# Función para obtener videos (JSON oficial de TikTok)
# -----------------------------
def get_videos(username, max_count=30):
    secUid = get_seguid(username)
    api_url = f"https://www.tiktok.com/api/post/item_list/?aid=1988&count={max_count}&secUid={secUid}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0 Safari/537.36"
    }
    r = requests.get(api_url, headers=headers)
    if r.status_code != 200:
        raise Exception(f"Error al obtener JSON: {r.status_code}")

    data = r.json()
    if "itemList" not in data:
        raise Exception("No se encontraron videos (TikTok bloqueó o usuario sin publicaciones)")
    return data["itemList"]

# -----------------------------
# Función para descargar thumbnails
# -----------------------------
def descargar_thumbnails(username, max_count=30):
    videos = get_videos(username, max_count)

    carpeta = f"{username}_thumbnails"
    os.makedirs(carpeta, exist_ok=True)

    for idx, video in enumerate(tqdm(videos, desc=f"Descargando {username}")):
        try:
            thumb_url = video["video"]["cover"]  # portada normal
            ruta = os.path.join(carpeta, f"thumb_{idx+1}.jpg")
            img = requests.get(thumb_url, stream=True)
            with open(ruta, "wb") as f:
                for chunk in img.iter_content(1024):
                    f.write(chunk)
        except Exception as e:
            print(f"⚠ Error con video {idx+1}: {e}")

    print(f"✅ Thumbnails descargados en: {carpeta}")

# -----------------------------
# EJECUTAR
# -----------------------------
usuario = "starempleo.uio"  # 👈 Cambia el usuario aquí
descargar_thumbnails(usuario, max_count=50)  # hasta 50 videos


JSONDecodeError: Expecting value: line 1 column 1 (char 0)